In [1]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints, keypoints_post_inference_processing
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os
import copy
import json

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
detection_model = RetinaNet().eval().cuda()
detection_model.load(DetConfig.trained_model_path)

In [3]:
keypoint_model = SegmentationModel().eval().cuda()
keypoint_model.load(KeyConfig.trained_model_path)

<All keys matched successfully>

In [4]:
image_transform = T.Compose([T.ToTensor()])

In [5]:
!rm -rf /code/data/datasets/temp_imgs/*


In [6]:
!rm -rf /code/data/datasets/keypoints/predictions/*

In [7]:
!rm -rf /code/data/datasets/detection/predictions/*

In [23]:
!rm -rf /code/data/datasets/common_hall/detection-keypoint-inference/*

In [9]:
image_paths = list(Config.images_directory.glob("*.jpg"))
random.shuffle(image_paths)
score_threshold: float = 0.5
# image_paths = image_paths[:5]
for image_path in tqdm(image_paths):
    slug = slugify(image_path)
    img = imageio.imread(image_path)
    img_h, img_w, _ = img.shape
    aug_boxes = detection_model(img)
    aug_boxes = aug_boxes[aug_boxes.scores > score_threshold].to_relative().to_tlbr()
    boxes = aug_boxes.array[:,:4]
    keypoints_list = []
    for box in boxes:
        x1, y1, x2, y2 = (box * np.array([img_w, img_h, img_w, img_h])).astype(int)
        roi_w = x2 - x1
        roi_h = y2 - y1
        roi_resized = crop_and_resize(box, img, KeyConfig.image_crop_size[0],KeyConfig.image_crop_size[1])
        roi_resized_w, roi_resized_h = roi_resized.size
        x = image_transform(roi_resized)
        x = torch.unsqueeze(x, 0).cuda()
        keypoints = keypoint_model(x)['keypoints'][0].detach().cpu().numpy()
        keypoints = keypoints_post_inference_processing(
            keypoints, roi_resized_w, roi_resized_h, roi_w, roi_h, x1, y1
        )
        keypoints_list.append(keypoints.astype(float).tolist())
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), thickness=4)
        img = cv2.circle(img, (int(keypoints[0][0]), int(keypoints[0][1])), radius=5, color=(0, 0, 255), thickness=-1) #Blue: Backtire
        img = cv2.circle(img, (int(keypoints[1][0]), int(keypoints[1][1])), radius=5, color=(255, 0, 0), thickness=-1) #Red Front tire
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    save_path = '/code/data/datasets/temp_imgs'
    filename = str(slug) + ".jpg"
    # box_path = Path("/code/data/datasets/detection/predictions")
    cv2.imwrite(os.path.join(save_path, filename), im_rgb)
    aug_boxes.to_darwin_file(
                output_path =  DetConfig.predictions_directory / (slug + ".json"),
                filename = filename,
                label_names = ["vehicle"],
            )
    with open(KeyConfig.predictions_directory / f"{slug}.json", "w") as f:
        f.write(json.dumps(keypoints_list))
        

    # boxes.to_file(get_prediction_path(slug))

  0%|          | 0/600 [00:00<?, ?it/s]/tmp/ipykernel_266336/302179755.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image_path)
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 600/600 [02:55<00:00,  3.42it/s]


In [24]:
multi_model_path = "/code/data/datasets/common_hall/detection-keypoint-inference"
existing_list = os.listdir("/root/.darwin/datasets/sparrow-computing/kj_speedtrap/releases/allv1/annotations")
detection_annotation_files = os.listdir(DetConfig.predictions_directory)
for detection_annotation_file in detection_annotation_files:
    if detection_annotation_file not in existing_list:
        detection_file = open(DetConfig.predictions_directory/detection_annotation_file)
        detection_annotation_content = json.load(detection_file)
        if (KeyConfig.predictions_directory/detection_annotation_file).exists():
            keypoint_file = open(KeyConfig.predictions_directory/detection_annotation_file)
            keypoint_annotation_content = json.load(keypoint_file)
            # for idx, tire in enumerate(KeyConfig.keypoint_names):
            #     key_dict = {}
            #     key_dict["name"] = tire
            #     key_dict["keypoint"] = {"x": keypoint_annotation_content[idx][0], "y": keypoint_annotation_content[idx][1]}
            #     detection_annotation_content['annotations'].append(key_dict)
            for key_pair in keypoint_annotation_content:
                for idx, tire in enumerate(KeyConfig.keypoint_names):
                    key_dict = {}
                    x, y = key_pair[idx]
                    key_dict["name"] =  tire
                    key_dict["keypoint"] = {
                        "x": x, "y":y
                    }
                    # key_dict["x"] =  x
                    # key_dict["y"] = y
                    detection_annotation_content['annotations'].append(key_dict)
        if len(detection_annotation_content["annotations"]) > 0:
            with open(Path(multi_model_path)/detection_annotation_file, "w") as f:
                f.write(json.dumps(detection_annotation_content))
        else:
            print(detection_annotation_file, "Did not have any predictions")

        

WBuBqS9h8_35559.json Did not have any predictions
WBuBqS9h8_33146.json Did not have any predictions
WBuBqS9h8_50410.json Did not have any predictions
WBuBqS9h8_51004.json Did not have any predictions
WBuBqS9h8_29341.json Did not have any predictions
WBuBqS9h8_53284.json Did not have any predictions
WBuBqS9h8_53024.json Did not have any predictions
WBuBqS9h8_29312.json Did not have any predictions
WBuBqS9h8_4513.json Did not have any predictions
WBuBqS9h8_53311.json Did not have any predictions
WBuBqS9h8_53418.json Did not have any predictions
WBuBqS9h8_49399.json Did not have any predictions
WBuBqS9h8_33161.json Did not have any predictions
WBuBqS9h8_35539.json Did not have any predictions
WBuBqS9h8_53148.json Did not have any predictions
WBuBqS9h8_35582.json Did not have any predictions
WBuBqS9h8_151.json Did not have any predictions
WBuBqS9h8_35600.json Did not have any predictions
WBuBqS9h8_50896.json Did not have any predictions


In [21]:
detection_annotation_content['annotations']

[{'bounding_box': {'x': 602.1806030273438,
   'y': 345.3334655761719,
   'w': 230.27789306640625,
   'h': 132.62420654296875},
  'name': 'vehicle'},
 {'bounding_box': {'x': 892.2577514648438,
   'y': 346.2511291503906,
   'w': 372.38116455078125,
   'h': 294.5596618652344},
  'name': 'vehicle'},
 {'bounding_box': {'x': 456.66058349609375,
   'y': 338.255859375,
   'w': 68.503173828125,
   'h': 80.26443481445312},
  'name': 'vehicle'},
 {'name': 'back_tire',
  'keypoint': {'x': 605.3202099737533, 'y': 459.26356589147287}},
 {'name': 'front_tire',
  'keypoint': {'x': 655.727034120735, 'y': 472.90697674418607}},
 {'name': 'back_tire',
  'keypoint': {'x': 920.8031496062993, 'y': 564.7906976744187}},
 {'name': 'front_tire',
  'keypoint': {'x': 1118.51968503937, 'y': 624.8062015503876}},
 {'name': 'back_tire',
  'keypoint': {'x': 467.0472440944882, 'y': 354.95090439276487}},
 {'name': 'front_tire', 'keypoint': {'x': 456.0, 'y': 338.0}}]